In [1]:
#!pip freeze

absl-py==2.0.0
ale-py==0.8.1
alembic==1.12.0
anyio @ file:///C:/ci/anyio_1644463705902/work/dist
argon2-cffi @ file:///C:/ci/argon2-cffi_1613038019788/work
attrs @ file:///C:/b/abs_09s3y775ra/croot/attrs_1668696195628/work
AutoROM==0.6.1
AutoROM.accept-rom-license==0.6.1
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
beautifulsoup4 @ file:///C:/ci/beautifulsoup4_1650292996413/work
bleach @ file:///opt/conda/conda-bld/bleach_1641577558959/work
brotlipy==0.7.0
cachetools==5.3.1
certifi @ file:///C:/b/abs_85o_6fm0se/croot/certifi_1671487778835/work/certifi
cffi @ file:///C:/b/abs_49n3v2hyhr/croot/cffi_1670423218144/work
charset-normalizer==3.3.0
click==8.1.7
cloudpickle==1.6.0
colorama @ file:///C:/b/abs_a9ozq0l032/croot/colorama_1672387194846/work
colorlog==6.7.0
comm==0.1.4
cryptography @ file:///C:/b/abs_8ecplyc3n2/croot/cryptography_1677533105000/work
cycler==0.11.0
debugpy @ file:///C:/ci/debugpy_1637091911212/work
decorator @ file:///opt/conda/conda-bld/decorator_

In [2]:
#!pip install gym==0.21.0

In [3]:
import gym # used to create observation space
import retro # used to run Street Fighter
import time # used to slow down Street Fighter

In [5]:
# create an environment with Street Fighter
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')

In [6]:
# sample of the observation space
env.observation_space.sample()

array([[[221, 143,  92],
        [ 18, 221, 166],
        [120, 122, 111],
        ...,
        [238,  69,  97],
        [132, 141,  87],
        [169,  44, 176]],

       [[ 16,   2,  82],
        [ 17, 234, 170],
        [ 22,  46,  12],
        ...,
        [108, 245,  59],
        [221, 220, 101],
        [ 76, 252, 106]],

       [[ 38, 224, 209],
        [ 87, 190, 105],
        [234, 234, 177],
        ...,
        [ 50,  73, 114],
        [179, 191,  57],
        [ 89, 202, 136]],

       ...,

       [[ 84,  30, 239],
        [ 85,  16, 118],
        [143, 173, 161],
        ...,
        [  0, 224,  58],
        [ 99, 150, 233],
        [250, 207,  41]],

       [[135, 215, 100],
        [247, 105,  69],
        [  4,  91, 255],
        ...,
        [195, 160, 109],
        [180,  40,  30],
        [ 69,  17, 164]],

       [[ 62,  12,  13],
        [228,  45, 204],
        [107, 218,   3],
        ...,
        [202,  13,  31],
        [149, 123,  85],
        [213, 248, 150]]

In [7]:
# sample of available actions
env.action_space

MultiBinary(12)

In [8]:
# reset game
#obs = env.reset()
# set flag to false
#done = False
#for game in range(1):
#    while not done:
#        if done:
#            obs = env.reset()
#        env.render()
#        obs, reward, done, info = env.step(env.action_space.sample())
#        print(reward)

In [9]:
# close any exisiting environment
env.close()

In [10]:
#show game info
#info

## Preprocessing

In [11]:
#!pip install opencv-python
#!pip install matplotlib

In [12]:
# import environment base class for a wrapper
from gym import Env
from gym import spaces
# import space shapes for env
from gym.spaces import MultiBinary
from gym.spaces import Box
# import numpy to calculate frame delta
import numpy as np
# import opencv for grayscaling
import cv2
#import matplotlib for plotting image
from matplotlib import pyplot as plt

In [13]:
# Create custom environment 
class StreetFighter(Env): 
    def __init__(self):
        super().__init__()
        # Specify action space and observation space 
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = gym.spaces.MultiBinary(12)
        # Startup and instance of the game 
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED)
    
    def reset(self):
        # Return the first frame 
        obs = self.game.reset()
        obs = self.preprocess(obs) 
        self.previous_frame = obs 
        
        # Create a attribute to hold the score delta 
        self.score = 0 
        return obs
    
    def preprocess(self, observation): 
        # Grayscaling to reduce data size
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Resize to reduce data size
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        # Add the channels value
        channels = np.reshape(resize, (84,84,1))
        return channels 
    
    def step(self, action): 
        # Take a step 
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs) 
        
        # Frame delta 
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs 
        
        # Reshape the reward function
        reward = info['score'] - self.score 
        self.score = info['score'] 
        
        return frame_delta, reward, done, info
    
    def render(self, *args, **kwargs):
        self.game.render()
        
    def close(self):
        self.game.close()

In [14]:
#plt.imshow(cv2.cvtColor(resize, cv2.COLOR_BGR2RGB))

In [15]:
#gray = cv2.cvtColor(obs, cv2.COLOR_BGR2GRAY)
#resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)

In [16]:
#env.close()

In [17]:
#env = StreetFighter()

In [18]:
#reset game
#obs = env.reset()
# set flag to false
#done = False
#for game in range(1):
#    while not done:
#        if done:
#            obs = env.reset()
#        env.render()
#        obs, reward, done, info = env.step(env.action_space.sample())
#        if reward > 0:
#            print(reward)

In [19]:
#obs = env.reset()

In [20]:
#obs, reward, done, info = env.step(env.action_space.sample())

In [21]:
#plt.imshow(cv2.cvtColor(obs,cv2.COLOR_BGR2RGB))

In [22]:
env.close()

## Hyperparameter Tuning

In [23]:
#!pip install --user torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [24]:
#!pip install --user stable-baselines3[extra] optuna
#!pip install ipywidgets

In [25]:
# Importing the optimzation frame - HPO
import optuna
# PPO algo for RL
from stable_baselines3 import PPO
# Bring in the eval policy method for metric calculation
from stable_baselines3.common.evaluation import evaluate_policy
# Import the sb3 monitor for logging 
from stable_baselines3.common.monitor import Monitor
# Import the vec wrappers to vectorize and frame stack
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
# Import os to deal with filepaths
import os

In [26]:
# create folders to hold logs
LOG_DIR = './logs/'
OPT_DIR = './opt/'

In [27]:
# Function to return test hyperparameters - define the object function
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [28]:
# create a savepath to hold the best model from the trial
SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(1))

In [29]:
# Run a training loop and return mean reward 
def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial) 
        # Create environment 
        env = StreetFighter()
        
        env = Monitor(env, LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')

        # Create algo 
        model = PPO('CnnPolicy', env, tensorboard_log = LOG_DIR, verbose=0, **model_params)
        model.learn(total_timesteps=100000)

        # Evaluate model 
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()

        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward

    except Exception as e:
        print(e)
        return -1000

In [30]:
#close any existing environments
env.close()

# Creating the experiment 
study = optuna.create_study(direction='maximize')
#study.optimize(optimize_agent, n_trials=10, n_jobs=1)
study.optimize(optimize_agent, n_trials=100, n_jobs=1)

[I 2023-10-19 18:09:45,753] A new study created in memory with name: no-name-a85cad50-93cc-4403-84f2-8160f19c37bb
C:\Users\Byngy\anaconda3\envs\myenv7\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  """
C:\Users\Byngy\anaconda3\envs\myenv7\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  
C:\Users\Byngy\anaconda3\envs\myenv7\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  import sys
C:\Users\Byngy\anaconda3\en

[I 2023-10-19 19:30:37,028] Trial 9 finished with value: 2000.0 and parameters: {'n_steps': 6410, 'gamma': 0.8397086779253398, 'learning_rate': 8.68605522388086e-05, 'clip_range': 0.16142028009628098, 'gae_lambda': 0.9498926733919661}. Best is trial 5 with value: 5800.0.
C:\Users\Byngy\anaconda3\envs\myenv7\lib\site-packages\stable_baselines3\ppo\ppo.py:138: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 3017`, after every 47 untruncated mini-batches, there will be a truncated mini-batch of size 9
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=3017 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2023-10-19 19:38:31,848] Trial 10 finished with value: 2000.0 and parameters: {'n_steps': 3017, 'gamma': 0.9045673773288537, 'learning_rate': 1.0049387535271707e-05, 'clip_range': 0.10862172657921446, 'gae_lambda': 0.8821901598077377}. Best is trial 5 wi

[I 2023-10-19 21:09:19,674] Trial 21 finished with value: 4400.0 and parameters: {'n_steps': 6901, 'gamma': 0.8718320153110676, 'learning_rate': 1.767356999831133e-05, 'clip_range': 0.1305043093354463, 'gae_lambda': 0.9033883668778929}. Best is trial 11 with value: 35500.0.
C:\Users\Byngy\anaconda3\envs\myenv7\lib\site-packages\stable_baselines3\ppo\ppo.py:138: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 8178`, after every 127 untruncated mini-batches, there will be a truncated mini-batch of size 50
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=8178 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2023-10-19 21:17:12,940] Trial 22 finished with value: 4000.0 and parameters: {'n_steps': 8178, 'gamma': 0.8907357394895902, 'learning_rate': 2.0001104587824316e-05, 'clip_range': 0.1316427077792816, 'gae_lambda': 0.9196569123611585}. Best is trial 

[I 2023-10-19 22:38:24,321] Trial 32 finished with value: 3500.0 and parameters: {'n_steps': 4766, 'gamma': 0.8736084369331921, 'learning_rate': 1.8313794651618588e-05, 'clip_range': 0.13984646480161955, 'gae_lambda': 0.889764301209183}. Best is trial 11 with value: 35500.0.
C:\Users\Byngy\anaconda3\envs\myenv7\lib\site-packages\stable_baselines3\ppo\ppo.py:138: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 5390`, after every 84 untruncated mini-batches, there will be a truncated mini-batch of size 14
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=5390 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2023-10-19 22:47:43,373] Trial 33 finished with value: 2000.0 and parameters: {'n_steps': 5390, 'gamma': 0.8680273617628577, 'learning_rate': 1.4608120510076981e-05, 'clip_range': 0.12393766344539735, 'gae_lambda': 0.8702882179233585}. Best is trial

[I 2023-10-20 00:12:37,189] Trial 43 finished with value: 18300.0 and parameters: {'n_steps': 3281, 'gamma': 0.8571153773151735, 'learning_rate': 1.3441742028545244e-05, 'clip_range': 0.15783828892285845, 'gae_lambda': 0.8152779015700781}. Best is trial 11 with value: 35500.0.
C:\Users\Byngy\anaconda3\envs\myenv7\lib\site-packages\stable_baselines3\ppo\ppo.py:138: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2736`, after every 42 untruncated mini-batches, there will be a truncated mini-batch of size 48
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2736 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2023-10-20 00:20:58,757] Trial 44 finished with value: 2700.0 and parameters: {'n_steps': 2736, 'gamma': 0.8652017330220662, 'learning_rate': 1.5257754033606289e-05, 'clip_range': 0.15196952773757486, 'gae_lambda': 0.8114855301831958}. Best is tri

...

[I 2023-10-20 03:49:26,038] Trial 65 finished with value: -1000.0 and parameters: {'n_steps': 4353, 'gamma': 0.8798775372326136, 'learning_rate': 2.4385872855500563e-05, 'clip_range': 0.17712401411782272, 'gae_lambda': 0.8358748787899097}. Best is trial 11 with value: 35500.0.


Expected parameter logits (Tensor of shape (64, 12)) of distribution Bernoulli(logits: torch.Size([64, 12])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

C:\Users\Byngy\anaconda3\envs\myenv7\lib\site-packages\stable_baselines3\ppo\ppo.py:138: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 5052`, after every 78 untruncated mini-batches, there will be a truncated mini-batch of size 60
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=5052 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2023-10-20 03:57:49,595] Trial 66 finished with value: 1700.0 and parameters: {'n_steps': 5052, 'gamma': 0.9014805610255593, 'learning_rate': 1.9933501987278385e-05, 'clip_range': 0.12609408497232755, 'gae_lambda': 0.8496204504429283}. Best is trial 11 with value: 35500.0.
C:\Users\Byngy\anaconda3\envs\myenv7\lib\site-packages\stable_baselines3\ppo\ppo.py:138: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 3303`, after every 51 untruncated mini-bat

[I 2023-10-20 10:16:36,130] Trial 77 finished with value: 2500.0 and parameters: {'n_steps': 3008, 'gamma': 0.8684634573822188, 'learning_rate': 1.2220361421634335e-05, 'clip_range': 0.1230658953064884, 'gae_lambda': 0.8245073680658099}. Best is trial 71 with value: 45900.0.
C:\Users\Byngy\anaconda3\envs\myenv7\lib\site-packages\stable_baselines3\ppo\ppo.py:138: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 3562`, after every 55 untruncated mini-batches, there will be a truncated mini-batch of size 42
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=3562 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2023-10-20 10:23:59,340] Trial 78 finished with value: 2500.0 and parameters: {'n_steps': 3562, 'gamma': 0.886137806535723, 'learning_rate': 1.1101057034365458e-05, 'clip_range': 0.138651512192008, 'gae_lambda': 0.8356870104621137}. Best is trial 71

[I 2023-10-20 11:59:05,826] Trial 89 finished with value: 2900.0 and parameters: {'n_steps': 2933, 'gamma': 0.8753457846099615, 'learning_rate': 1.1563935648939163e-05, 'clip_range': 0.11969498626160731, 'gae_lambda': 0.8789356548333359}. Best is trial 85 with value: 64200.0.
C:\Users\Byngy\anaconda3\envs\myenv7\lib\site-packages\stable_baselines3\ppo\ppo.py:138: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 3355`, after every 52 untruncated mini-batches, there will be a truncated mini-batch of size 27
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=3355 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2023-10-20 12:06:49,970] Trial 90 finished with value: 2000.0 and parameters: {'n_steps': 3355, 'gamma': 0.8840392657883439, 'learning_rate': 1.1063282576092375e-05, 'clip_range': 0.19232735618511002, 'gae_lambda': 0.8722327379635195}. Best is tria

In [31]:
# observe the best parameters
study.best_params

{'n_steps': 3654,
 'gamma': 0.8822182578312063,
 'learning_rate': 1.077200939126095e-05,
 'clip_range': 0.16765709372090606,
 'gae_lambda': 0.8556741461492272}

In [32]:
# observe the best trial
study.best_trial

FrozenTrial(number=85, state=TrialState.COMPLETE, values=[64200.0], datetime_start=datetime.datetime(2023, 10, 20, 11, 17, 51, 245693), datetime_complete=datetime.datetime(2023, 10, 20, 11, 27, 2, 620101), params={'n_steps': 3654, 'gamma': 0.8822182578312063, 'learning_rate': 1.077200939126095e-05, 'clip_range': 0.16765709372090606, 'gae_lambda': 0.8556741461492272}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_steps': IntDistribution(high=8192, log=False, low=2048, step=1), 'gamma': FloatDistribution(high=0.9999, log=True, low=0.8, step=None), 'learning_rate': FloatDistribution(high=0.0001, log=True, low=1e-05, step=None), 'clip_range': FloatDistribution(high=0.4, log=False, low=0.1, step=None), 'gae_lambda': FloatDistribution(high=0.99, log=False, low=0.8, step=None)}, trial_id=85, value=None)

In [33]:
# use the information above to load the best performing model
model = PPO.load(os.path.join(OPT_DIR, 'trial_85_best_model.zip'))

In [34]:
# Import base callback 
from stable_baselines3.common.callbacks import BaseCallback

class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
            
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [35]:
CHECKPOINT_DIR = './train/'

In [36]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [37]:
# Create environment 
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [38]:
model_params = study.best_params

In [39]:
model_params['n_steps'] = 7488  # set n_steps to 7488 or a factor of 64

In [40]:
# model_params['learning_rate'] = 5e-7
model_params

{'n_steps': 7488,
 'gamma': 0.8822182578312063,
 'learning_rate': 1.077200939126095e-05,
 'clip_range': 0.16765709372090606,
 'gae_lambda': 0.8556741461492272}

In [41]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [42]:
# Reload previous weights from HPO
model.load(os.path.join(OPT_DIR, 'trial_0_best_model.zip'))

In [43]:
# Kick off training 
model.learn(total_timesteps=5000000, callback=callback)
# model.learn(total_timestep=5000000) 

Logging to ./logs/PPO_234
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6.77e+03 |
|    ep_rew_mean     | 5.2e+03  |
| time/              |          |
|    fps             | 393      |
|    iterations      | 1        |
|    time_elapsed    | 19       |
|    total_timesteps | 7488     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 6.77e+03     |
|    ep_rew_mean          | 5.2e+03      |
| time/                   |              |
|    fps                  | 302          |
|    iterations           | 2            |
|    time_elapsed         | 49           |
|    total_timesteps      | 14976        |
| train/                  |              |
|    approx_kl            | 0.0030078518 |
|    clip_fraction        | 0.0389       |
|    clip_range           | 0.168        |
|    entropy_loss         | -8.32        |
|    explained_variance   | 0.00

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.02e+04    |
|    ep_rew_mean          | 2.81e+04    |
| time/                   |             |
|    fps                  | 253         |
|    iterations           | 11          |
|    time_elapsed         | 324         |
|    total_timesteps      | 82368       |
| train/                  |             |
|    approx_kl            | 0.008627513 |
|    clip_fraction        | 0.14        |
|    clip_range           | 0.168       |
|    entropy_loss         | -8.26       |
|    explained_variance   | 0.00732     |
|    learning_rate        | 1.08e-05    |
|    loss                 | 173         |
|    n_updates            | 100         |
|    policy_gradient_loss | 0.00132     |
|    value_loss           | 6.88e+04    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.71e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.02e+04    |
|    ep_rew_mean          | 3.29e+04    |
| time/                   |             |
|    fps                  | 249         |
|    iterations           | 21          |
|    time_elapsed         | 630         |
|    total_timesteps      | 157248      |
| train/                  |             |
|    approx_kl            | 0.020175826 |
|    clip_fraction        | 0.272       |
|    clip_range           | 0.168       |
|    entropy_loss         | -8.06       |
|    explained_variance   | 0.0542      |
|    learning_rate        | 1.08e-05    |
|    loss                 | 20.5        |
|    n_updates            | 200         |
|    policy_gradient_loss | 0.00432     |
|    value_loss           | 4.87e+04    |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.05e+04

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.05e+04   |
|    ep_rew_mean          | 3.32e+04   |
| time/                   |            |
|    fps                  | 247        |
|    iterations           | 31         |
|    time_elapsed         | 937        |
|    total_timesteps      | 232128     |
| train/                  |            |
|    approx_kl            | 0.04340727 |
|    clip_fraction        | 0.418      |
|    clip_range           | 0.168      |
|    entropy_loss         | -7.63      |
|    explained_variance   | 0.242      |
|    learning_rate        | 1.08e-05   |
|    loss                 | 19.3       |
|    n_updates            | 300        |
|    policy_gradient_loss | 0.0161     |
|    value_loss           | 4.74e+04   |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.03e+04    |
|    ep_rew_m

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.03e+04   |
|    ep_rew_mean          | 2.9e+04    |
| time/                   |            |
|    fps                  | 247        |
|    iterations           | 41         |
|    time_elapsed         | 1242       |
|    total_timesteps      | 307008     |
| train/                  |            |
|    approx_kl            | 0.02063616 |
|    clip_fraction        | 0.262      |
|    clip_range           | 0.168      |
|    entropy_loss         | -7.17      |
|    explained_variance   | 0.601      |
|    learning_rate        | 1.08e-05   |
|    loss                 | 8.46e+03   |
|    n_updates            | 400        |
|    policy_gradient_loss | 0.0014     |
|    value_loss           | 2.21e+03   |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.02e+04    |
|    ep_rew_m

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.55e+03    |
|    ep_rew_mean          | 1.85e+03    |
| time/                   |             |
|    fps                  | 140         |
|    iterations           | 621         |
|    time_elapsed         | 33052       |
|    total_timesteps      | 4650048     |
| train/                  |             |
|    approx_kl            | 0.007985556 |
|    clip_fraction        | 0.0713      |
|    clip_range           | 0.168       |
|    entropy_loss         | -0.266      |
|    explained_variance   | 0.0289      |
|    learning_rate        | 1.08e-05    |
|    loss                 | 16.8        |
|    n_updates            | 6200        |
|    policy_gradient_loss | 0.0011      |
|    value_loss           | 351         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.54e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.55e+03    |
|    ep_rew_mean          | 1.9e+03     |
| time/                   |             |
|    fps                  | 140         |
|    iterations           | 631         |
|    time_elapsed         | 33557       |
|    total_timesteps      | 4724928     |
| train/                  |             |
|    approx_kl            | 0.010028125 |
|    clip_fraction        | 0.0707      |
|    clip_range           | 0.168       |
|    entropy_loss         | -0.214      |
|    explained_variance   | 0.123       |
|    learning_rate        | 1.08e-05    |
|    loss                 | 0.667       |
|    n_updates            | 6300        |
|    policy_gradient_loss | 0.000583    |
|    value_loss           | 306         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.56e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.57e+03    |
|    ep_rew_mean          | 1.97e+03    |
| time/                   |             |
|    fps                  | 140         |
|    iterations           | 641         |
|    time_elapsed         | 34067       |
|    total_timesteps      | 4799808     |
| train/                  |             |
|    approx_kl            | 0.012809329 |
|    clip_fraction        | 0.0734      |
|    clip_range           | 0.168       |
|    entropy_loss         | -0.245      |
|    explained_variance   | 0.108       |
|    learning_rate        | 1.08e-05    |
|    loss                 | 0.483       |
|    n_updates            | 6400        |
|    policy_gradient_loss | 0.00207     |
|    value_loss           | 581         |
-----------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 5.57e+03  

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5.58e+03     |
|    ep_rew_mean          | 2.01e+03     |
| time/                   |              |
|    fps                  | 140          |
|    iterations           | 651          |
|    time_elapsed         | 34580        |
|    total_timesteps      | 4874688      |
| train/                  |              |
|    approx_kl            | 0.0076071555 |
|    clip_fraction        | 0.0704       |
|    clip_range           | 0.168        |
|    entropy_loss         | -0.247       |
|    explained_variance   | 0.0412       |
|    learning_rate        | 1.08e-05     |
|    loss                 | 1.02         |
|    n_updates            | 6500         |
|    policy_gradient_loss | 0.000512     |
|    value_loss           | 623          |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.52e+03    |
|    ep_rew_mean          | 1.96e+03    |
| time/                   |             |
|    fps                  | 140         |
|    iterations           | 661         |
|    time_elapsed         | 35140       |
|    total_timesteps      | 4949568     |
| train/                  |             |
|    approx_kl            | 0.010970419 |
|    clip_fraction        | 0.0701      |
|    clip_range           | 0.168       |
|    entropy_loss         | -0.242      |
|    explained_variance   | 0.259       |
|    learning_rate        | 1.08e-05    |
|    loss                 | 0.795       |
|    n_updates            | 6600        |
|    policy_gradient_loss | 0.0013      |
|    value_loss           | 783         |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5.52

In [68]:
model = PPO.load('./train/best_model_5000000.zip')

In [66]:
env.close()

In [69]:
mean_reward, _ = evaluate_policy(model, env, render=True, n_eval_episodes=5)

AttributeError: 'RetroEnv' object has no attribute 'em'

In [61]:
mean_reward

2300.0

In [54]:
obs = env.reset()

In [62]:
obs.shape

(1, 84, 84, 4)

In [70]:
env.step(model.predict(obs)[0])

AttributeError: 'RetroEnv' object has no attribute 'em'

In [71]:
env.close()
# Reset game to starting state
obs = env.reset()
# Set flag to flase
done = False
for game in range(1):
    print('Score:')
    while not done: 
        if done: 
            print('Game Finished')
            obs = env.reset()
        env.render()
        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)
        time.sleep(0.01)
        print(reward)

AttributeError: 'RetroEnv' object has no attribute 'em'